# KNN代码实现与数据分析案例
## 代码实现

### 基本原理
1. 构造一个n维空间，空间内每个向量X有对应的标签Y，输入一个预测点x，取在一种距离度量下（一般为p2范数），空间内距离x最近的k个点，再观察这几个点的标签值y，通过一个函数（常用的实际上就是找这n个标签里面最多的一个yi作为预测标签），确定这个预测点x的预测标签y
2. 下面通过python写一遍算法，增强理解，再用scikit-learn库的KNeighborsClassifier实现
3. 优化KNN，可以用KD树算法，快速搜索到最近的k个点

In [101]:
#距离的度量——范数（norm）
import math

def L(x,y,p=2):
    if len(x) == len(y) and len(x):
        sum = 0
        for i in range(len(x)):
            sum += math.pow(abs(x[i]-y[i]),p)
        return math.pow(sum,1/p)
    else:
        return 0

In [4]:
#导库

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline        
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from collections import Counter

In [2]:
#导入数据初步研究

iris = load_iris()
df = pd.DataFrame(iris.data , columns=iris.feature_names)
df['label'] = iris.target
df.columns = ['sepal length','sepal width','petal length','petal width','label']
df.describe()


,sepal length,sepal width,petal length,petal width,label
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


### 简单取出二维的数据集，绘制散点图，直观感受

In [5]:
plt.scatter(df[:50]['sepal length'], df[:50]['sepal width'], label='0')
plt.scatter(df[50:100]['sepal length'], df[50:100]['sepal width'], label='1')
plt.scatter(df[100:150]['sepal length'],df[100:150]['sepal width'],label = '2')
plt.xlabel('sepal length')
plt.ylabel('sepal width')
plt.legend()

In [105]:
data = np.array(df)     
X, y = data[:,:-1], data[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)   #拆分测试集与训练集


In [106]:
class KNN:
    def __init__(self, X_train, y_train, k=3, p=2):
        self.k = k
        self.p = p
        self.X_train = X_train
        self.y_train = y_train
    def predict(self,x):
        totl_ls = [(np.linalg.norm(x - self.X_train[i]),self.y_train[i]) for i in range(len(self.X_train))]
        k_ls = sorted(totl_ls, key=lambda x: x[0])[:self.k]
        labels = [k[-1] for k in k_ls]
        counter = Counter(labels)
        max_label = sorted(counter.items(), key=lambda x: x[1])[-1][0]
        return max_label
    
    def score(self, X_test, y_test):
        right_count = 0
        for X, y in zip(X_test, y_test):
            label = self.predict(X)
            if label == y:
                right_count += 1
        return right_count / len(X_test)
        

### 用手搓KNN分类器预测

In [107]:
results = []
for i in range(1, 5):
    for s in range(4, 10):
        clf = KNN(X_train, y_train, k=s ,p=i)
        score = clf.score(X_test, y_test)
        results.append((i, s, score))

results


[(1, 4, 0.9666666666666667),
 (1, 5, 0.9666666666666667),
 (1, 6, 1.0),
 (1, 7, 1.0),
 (1, 8, 1.0),
 (1, 9, 1.0),
 (2, 4, 0.9666666666666667),
 (2, 5, 0.9666666666666667),
 (2, 6, 1.0),
 (2, 7, 1.0),
 (2, 8, 1.0),
 (2, 9, 1.0),
 (3, 4, 0.9666666666666667),
 (3, 5, 0.9666666666666667),
 (3, 6, 1.0),
 (3, 7, 1.0),
 (3, 8, 1.0),
 (3, 9, 1.0),
 (4, 4, 0.9666666666666667),
 (4, 5, 0.9666666666666667),
 (4, 6, 1.0),
 (4, 7, 1.0),
 (4, 8, 1.0),
 (4, 9, 1.0)]

## 基于scikit-learn库的实现

In [108]:
from sklearn.neighbors import KNeighborsClassifier

In [109]:

clf_sk = KNeighborsClassifier()
clf_sk.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [110]:
clf_sk.score(X_test, y_test)

0.9666666666666667

## 构造KD树

In [111]:
class KdNode(object):
    def __init__(self, real_node, split_dim, left, right):
        self.node = real_node  # k维向量实际点作为节点
        self.split_dim = split_dim  # 按照split_dim : int这个维度切割
        self.left = left  # 节点左侧，split_dim维度上值小于dot
        self.right = right  # 反之

class KdTree(object):
    def __init__(self, data):
        k = len(data[0])  # 数据的维度

        def CreateNode(split_dim, data_set):  # 按第split_dim维划分数据集创建KdNode
            if not data_set:  
                return None
            
            data_set.sort(key=lambda x: x[split_dim])
            
            median = data_set[len(data_set) // 2]  # 中位数分割点
            split_next = (split_dim + 1) % k  # 指定下一个分割维度的规则（模型确定）

            # 递归的创建左右子树，形成完整的KD树
            return KdNode(
                median,
                split_dim,
                CreateNode(split_next, data_set[:len(data_set) // 2]),  
                CreateNode(split_next, data_set[len(data_set) // 2 + 1:]))  

        self.root = CreateNode(0, data)  # 从第0维分量开始构建kd树,返回值为根节点
        
def preorder(root):     #遍历kd树
    print(root.node)
    if root.left:  
        preorder(root.left)
    if root.right:
        preorder(root.right)


## KD树搜索

In [112]:
from math import sqrt
from collections import namedtuple

# 定义一个namedtuple,分别存放最近坐标点、最近距离和访问过的节点数
result = namedtuple("Result_tuple",
                    "nearest_point  nearest_dist  nodes_visited")


def find_nearest(tree, point):
    k = len(point)  # 数据维度

    def travel(kd_node, target, max_dist):     #max_dist最大搜索半径
        if kd_node is None:
            return result([0] * k, float("inf"),
                          0)  # python中用float("inf")和float("-inf")表示正负无穷

        nodes_visited = 1

        s = kd_node.split_dim  # 进行分割的维度
        pivot_node = kd_node.node  # 分割的实值节点（分割的超平面）

        if target[s] <= pivot_node[s]:  # 如果目标点第s维小于分割轴的对应值(目标离左子树更近)
            nearer_node = kd_node.left  # 下一个访问节点为左子树根节点
            further_node = kd_node.right  # 同时记录下较远的右子树
        else:  # 目标离右子树更近
            nearer_node = kd_node.right  # 下一个访问节点为右子树根节点
            further_node = kd_node.left

        temp1 = travel(nearer_node, target, max_dist)  # 进行遍历找到包含目标点的区域，先找到目标点存在的最深的节点

        nearest = temp1.nearest_point  # 以此最深的叶结点作为“当前最近点”，然后不断回溯（最大搜索半径判断是否需要剪枝），看最近距离是否需要更新
        dist = temp1.nearest_dist  # 更新最近距离

        nodes_visited += temp1.nodes_visited      #记录回溯的节点个数，按道理是少于逐个遍历的

        if dist < max_dist:     #更新的最近距离dist是否在最大搜索半径内
            max_dist = dist  # 如果在，最近点将在以目标点为球心，最大搜索半径max_dist（已更新）为半径的超球体内

        temp_dist = abs(pivot_node[s] - target[s])  # 第s维上目标点与分割超平面的距离
        if max_dist < temp_dist:  # 判断超球体是否与超平面相交
            return result(nearest, dist, nodes_visited)  # 不相交则可以剪枝，没有必要继续算远一点的平行节点空间内的距离，保持最大搜索半径（肯定小于与超平面的距离，也必然小于真实的p2范数距离）

      
        # 不可剪枝则计算目标点与分割点的欧氏距离
        temp_dist = sqrt(sum((p1 - p2)**2 for p1, p2 in zip(pivot_node, target)))

        if temp_dist < dist:  # 如果距离比最大搜索半径“更近”
            nearest = pivot_node  #则更新最大搜索半径和当前最近距离
            dist = temp_dist  
            max_dist = dist  

        # 检查另一个子结点对应的区域是否有更近的点
        temp2 = travel(further_node, target, max_dist)

        nodes_visited += temp2.nodes_visited
        if temp2.nearest_dist < dist:  # 如果另一个子结点内存在更近距离
            nearest = temp2.nearest_point  # 更新最近点
            dist = temp2.nearest_dist  # 更新最近距离

        return result(nearest, dist, nodes_visited)

    return travel(tree.root, point, float("inf"))  # 从根节点开始递归

### 另一种实现

In [113]:
# 构建kd树，搜索待预测点所属区域
from collections import namedtuple
import numpy as np


# 建立节点类
class Node(namedtuple("Node", "location left_child right_child")):
    def __repr__(self):
        return str(tuple(self))


# kd tree类
class KdTree():
    def __init__(self, k=1):
        self.k = k
        self.kdtree = None

    # 构建kd tree
    def _fit(self, X, depth=0):
        try:
            k = self.k
        except IndexError as e:
            return None
        # 这里可以展开，通过方差选择axis
        axis = depth % k
        X = X[X[:, axis].argsort()]
        median = X.shape[0] // 2
        try:
            X[median]
        except IndexError:
            return None
        return Node(location=X[median],
                    left_child=self._fit(X[:median], depth + 1),
                    right_child=self._fit(X[median + 1:], depth + 1))

    def _search(self, point, tree=None, depth=0, best=None):
        if tree is None:
            return best
        k = self.k
        # 更新 branch
        if point[0][depth % k] < tree.location[depth % k]:
            next_branch = tree.left_child
        else:
            next_branch = tree.right_child
        if not next_branch is None:
            best = next_branch.location
        return self._search(point,
                            tree=next_branch,
                            depth=depth + 1,
                            best=best)

    def fit(self, X):
        self.kdtree = self._fit(X)
        return self.kdtree

    def predict(self, X):
        res = self._search(X, self.kdtree)
        return res

In [114]:
KNN = KdTree()
X_train = np.array([[2, 3], [5, 4], [9, 6], [4, 7], [8, 1], [7, 2]])
KNN.fit(X_train)
X_new = np.array([[3, 4.5]])
res = KNN.predict(X_new)

x1 = res[0]
x2 = res[1]

print("x点的最近邻点是({0}, {1})".format(x1, x2))

x点的最近邻点是(2, 3)
